In [ ]:
!pip install transformers torch torchvision pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.9/976.9 kB 19.3 MB/s eta 0:00:00


In [ ]:
import cv2
import torch
from PIL import Image
from transformers import AutoModelForImageClassification, AutoFeatureExtractor

# Load the model and feature extractor
model_name = "touchtech/fashion-images-gender-age-vit-large-patch16-224-in21k-v3"
model = AutoModelForImageClassification.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# Load Haarcascade Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load the image
image_path = "/content/th.jpeg"  # Update this if needed
image_cv = cv2.imread(image_path)
gray_image = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

# Labels for predictions (Mapping category numbers to Male/Female)
gender_labels = {
    0: "Female",  # Previously "Female Adult"
    1: "Female",  # Previously "Female Child"
    2: "Male",    # Previously "Male Adult"
    3: "Male",    # Previously "Male Child"
    4: "Unknown"  # Category 'Pack' (if present, ignore)
}

# Process each detected face
for (x, y, w, h) in faces:
    face = image_cv[y:y+h, x:x+w]  # Crop the face
    image_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))  # Convert to PIL format

    # Preprocess and predict
    inputs = feature_extractor(images=image_pil, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_label = predictions.argmax().item()
        confidence = predictions.max().item()

    # Ignore 'Unknown' category and only classify Male/Female
    if predicted_label != 4 and confidence > 0.6:
        label_text = f"{gender_labels[predicted_label]} ({confidence:.2f})"
        cv2.rectangle(image_cv, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image_cv, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Save or display the output image
output_path = "/mnt/data/output.png"
cv2.imwrite(output_path, image_cv)

# Display the output (Colab-friendly)
from google.colab.patches import cv2_imshow
cv2_imshow(image_cv)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
from ultralytics import YOLO
import cv2
import torch
from PIL import Image
from transformers import AutoModelForImageClassification, AutoFeatureExtractor

# Load YOLOv8 model (for person detection)
yolo_model = YOLO("yolov8n.pt")  # Use 'yolov8s.pt' for better accuracy

# Load Gender Classification Model
gender_model_name = "touchtech/fashion-images-gender-age-vit-large-patch16-224-in21k-v3"
gender_model = AutoModelForImageClassification.from_pretrained(gender_model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(gender_model_name)

# Load Image
image_path = "/content/group-of-business-people-scaled.webp"  # Update this if needed
image_cv = cv2.imread(image_path)

# Detect People using YOLO
results = yolo_model(image_path)
detections = results[0].boxes.data  # Get bounding boxes

# Define Gender Labels
gender_labels = {
    0: "Female", 1: "Female", 2: "Male", 3: "Male", 4: "Unknown"
}

# Process each detected person
for det in detections:
    x1, y1, x2, y2, conf, cls = det.tolist()

    # Only consider "Person" class (YOLO class 0)
    if int(cls) == 0:
        person_img = image_cv[int(y1):int(y2), int(x1):int(x2)]  # Crop person
        person_pil = Image.fromarray(cv2.cvtColor(person_img, cv2.COLOR_BGR2RGB))

        # Predict Gender
        inputs = feature_extractor(images=person_pil, return_tensors="pt")
        with torch.no_grad():
            outputs = gender_model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_label = predictions.argmax().item()
            confidence = predictions.max().item()

        # Draw Bounding Box with Gender
        if predicted_label != 4 and confidence > 0.6:
            label_text = f"{gender_labels[predicted_label]} ({confidence:.2f})"
            cv2.rectangle(image_cv, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image_cv, label_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Save & Display the Output
output_path = "/mnt/data/output.png"
cv2.imwrite(output_path, image_cv)

from google.colab.patches import cv2_imshow
cv2_imshow(image_cv)


In [ ]:
#same code for video data
import cv2
import torch
import os
import shutil
from ultralytics import YOLO
from transformers import AutoModelForImageClassification, AutoFeatureExtractor
from PIL import Image
import numpy as np

# Load YOLO model (for person detection)
yolo_model = YOLO("yolov8n.pt")  # Use 'yolov8s.pt' for better accuracy

# Load Gender Classification Model
gender_model_name = "touchtech/fashion-images-gender-age-vit-large-patch16-224-in21k-v3"
gender_model = AutoModelForImageClassification.from_pretrained(gender_model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(gender_model_name)

# Define Gender Labels (0 & 1 only for male/female)
gender_labels = {
    0: "Female",
    1: "Female",
    2: "Male",
    3: "Male",
    4: "Unknown"
}

# Input & Output Paths
video_path = "/content/drive/MyDrive/sample_video-crowd.mp4"  # Change this to your video file
output_video_path ="/output_video.mp4"

# Create temp directory for frames
frame_dir = "/mnt/data/frames"
if os.path.exists(frame_dir):
    shutil.rmtree(frame_dir)
os.makedirs(frame_dir)

# Load Video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO for person detection
    results = yolo_model(frame)
    detections = results[0].boxes.data  # Bounding boxes

    for det in detections:
        x1, y1, x2, y2, conf, cls = det.tolist()

        # Only process "Person" class (YOLO class 0)
        if int(cls) == 0:
            person_img = frame[int(y1):int(y2), int(x1):int(x2)]
            person_pil = Image.fromarray(cv2.cvtColor(person_img, cv2.COLOR_BGR2RGB))

            # Predict Gender
            inputs = feature_extractor(images=person_pil, return_tensors="pt")
            with torch.no_grad():
                outputs = gender_model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predicted_label = predictions.argmax().item()
                confidence = predictions.max().item()

            # Draw bounding box with label
            if predicted_label != 4 and confidence > 0.6:
                label_text = f"{gender_labels[predicted_label]} ({confidence:.2f})"
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, label_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    frame_filename = os.path.join(frame_dir, f"frame_{frame_count:05d}.png")
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

output_video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

for i in range(frame_count):
    frame = cv2.imread(os.path.join(frame_dir, f"frame_{i:05d}.png"))
    output_video.write(frame)

output_video.release()
print(f"✅ Process Complete! Output saved at {output_video_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 103MB/s]



0: 384x640 10 persons, 114.0ms
Speed: 9.7ms preprocess, 114.0ms inference, 377.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 9.5ms
Speed: 2.4ms preprocess, 9.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 10.3ms
Speed: 2.4ms preprocess, 10.3ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 9.4ms
Speed: 2.2ms preprocess, 9.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 10.3ms
Speed: 2.2ms preprocess, 10.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 9.1ms
Speed: 2.3ms preprocess, 9.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 10.6ms
Speed: 3.5ms preprocess, 10.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 chair, 9.5ms
Speed: 3.6ms preprocess, 9.5ms inference, 1.5ms postprocess per ima

KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("/output_video.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(model.config.id2label)


{0: 'model-female-adult', 1: 'model-female-child', 2: 'model-male-adult', 3: 'model-male-child', 4: 'pack'}


In [ ]:
#working good

from transformers import ViTImageProcessor, ViTForImageClassification
import torch
import cv2
from PIL import Image

model_name = "touchtech/fashion-images-gender-age-vit-large-patch16-224-in21k-v3"
model = ViTForImageClassification.from_pretrained(model_name)
processor = ViTImageProcessor.from_pretrained(model_name)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

video_path = "/content/youtube_video.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    frame_count += 1
    if frame_count % 5 != 0:  # Process every 5th frame for better accuracy
        continue

    # Convert frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]  # Crop the face region
        image = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

        labels = {0: "Female Adult", 1: "Female Child", 2: "Male Adult", 3: "Male Child", 4: "Pack"}
        predicted_label = predictions.argmax().item()
        confidence = predictions.max().item()

        if confidence > 0.6:
            print(f"Frame {frame_count}: Predicted Gender → {labels[predicted_label]} (Confidence: {confidence:.2f})")

cap.release()
#cv2.destroyAllWindows()

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
import torch
import cv2
from PIL import Image

# Load the pretrained model
model_name = "touchtech/fashion-images-gender-age-vit-large-patch16-224-in21k-v3"
model = ViTForImageClassification.from_pretrained(model_name)
processor = ViTImageProcessor.from_pretrained(model_name)

# Load face detection model (Haarcascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open video file
video_path = "/content/202134-917314709_small.mp4"  # Change this to your actual video path
cap = cv2.VideoCapture(video_path)

# Model labels (Filtered: Only Male and Female)
filtered_labels = {0: "Female", 2: "Male"}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    frame_count += 1
    if frame_count % 5 != 0:  # Process every 5th frame for efficiency
        continue

    # Convert frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]  # Crop the face region

        # Ensure face region is valid
        if face.size == 0:
            continue

        image = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))  # Convert to PIL Image

        # Preprocess the face
        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

        # Get predicted label and confidence
        predicted_label = predictions.argmax().item()
        confidence = predictions.max().item()

        # Only consider Male (2) and Female (0) with confidence > 0.6
        if predicted_label in filtered_labels and confidence > 0.7:
            print(f"Frame {frame_count}: Predicted Gender → {filtered_labels[predicted_label]} (Confidence: {confidence:.2f})")

cap.release()


Frame 5: Predicted Gender → Female (Confidence: 1.00)
Frame 10: Predicted Gender → Female (Confidence: 1.00)
Frame 10: Predicted Gender → Male (Confidence: 0.89)
Frame 10: Predicted Gender → Female (Confidence: 0.99)
Frame 15: Predicted Gender → Female (Confidence: 1.00)
Frame 20: Predicted Gender → Female (Confidence: 1.00)
Frame 25: Predicted Gender → Female (Confidence: 1.00)
Frame 30: Predicted Gender → Female (Confidence: 1.00)
Frame 35: Predicted Gender → Female (Confidence: 1.00)
Frame 40: Predicted Gender → Female (Confidence: 1.00)
Frame 45: Predicted Gender → Female (Confidence: 1.00)
Frame 50: Predicted Gender → Female (Confidence: 1.00)
Frame 55: Predicted Gender → Female (Confidence: 1.00)
Frame 60: Predicted Gender → Female (Confidence: 1.00)
Frame 65: Predicted Gender → Female (Confidence: 1.00)
Frame 70: Predicted Gender → Female (Confidence: 0.99)
Frame 75: Predicted Gender → Female (Confidence: 0.99)
Frame 80: Predicted Gender → Female (Confidence: 0.99)
Frame 85: Pre